In [1]:
# Import libraries
import numpy as np
import os
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import h5py
import mne
import pyedflib

mne.set_log_level('WARNING')

In [4]:
# Create dataset to save the training sample

# TRAINING

# Training samples
f = h5py.File("X_training_lcomp.h5","a")
training_dset = f.create_dataset('training', shape=(1800,32,32,3), maxshape=(183600,32,32,3), compression="gzip", compression_opts=4)

# Training labels
f = h5py.File("Y_training_lcomp.h5","a")
traininglbs_dset = f.create_dataset('train_labels', shape=(1800,1), maxshape=(183600,1), compression="gzip", compression_opts=4)

image_index = 0

round_labels = np.zeros((300,1))
label_index = 0
subject_number = 1

In [6]:
# Electrodes coordinates

# Get example file to extract electrode coordinates
path = "../../Dataset/S001"                                  # Read specific subject dataset
file_name = os.path.join(path, 'S001R04.edf')             # Read specific round of the subject

round_data = mne.io.read_raw_edf(file_name)
round_data.load_data()
round_data.rename_channels(lambda s: s.strip("."))

montage_kind = "standard_1020"
montage = mne.channels.make_standard_montage(montage_kind)

round_data.set_montage(montage, match_case=False)
round_data.set_eeg_reference("average")                    # ADD THIS IN THE LOOP AS WELL

montage_image = round_data.get_montage()
a = getattr(montage_image,'dig')

# Get coordinates of electrodes in a single array
elec_coord = np.zeros((67,2))

# Information about electrodes
elec_info = getattr(montage_image,'dig')

# Save x and y coordinates of LPA(index 0), nasion(index 1), RPA(index 2) and all electrodes (indeces 3-67) in the array
for i in range(0,67):
    xyz = elec_info[i].get('r')
    elec_coord[i] = [xyz[1],xyz[0]]

# Normalize coordinates
max_y = np.amax(elec_coord[:,0])
min_y = np.amin(elec_coord[:,0])

max_x = np.amax(elec_coord[:,1])
min_x = np.amin(elec_coord[:,1])

for i in range(0,67):
    
    # Normalize y coordinates
    # Positive
    if (elec_coord[i][0] > 0):
        elec_coord[i][0] = abs(round(16*abs(elec_coord[i][0]/max_y))-16)
    # Negative
    else:
        elec_coord[i][0] = round(15*abs(elec_coord[i][0]/(min_y)))+16
    
    # Normanize x coordinates
    # Positive
    if (elec_coord[i][1] > 0):
        elec_coord[i][1] = round(15*abs(elec_coord[i][1]/max_x))+16
    # Negative
    else:
        elec_coord[i][1] = abs(round(16*abs(elec_coord[i][1]/min_x))-16)

print(elec_coord)
montage_image.save("montage.fif")

# Electrode coordinates used to assign values and interpolate later
electrodes = np.zeros((64,2))

for k in range(0,64):
    electrodes[k] = elec_coord[k+3]

image_x, image_y = np.mgrid[0:32, 0:32]
interpolation = np.zeros((10,32,32,3))

[[16.  0.]
 [ 1. 16.]
 [16. 31.]
 [10.  2.]
 [ 9.  5.]
 [ 8. 10.]
 [ 8. 16.]
 [ 8. 22.]
 [ 9. 27.]
 [ 9. 30.]
 [14.  1.]
 [14.  4.]
 [13.  9.]
 [13. 16.]
 [13. 23.]
 [13. 28.]
 [14. 31.]
 [19.  1.]
 [19.  4.]
 [19.  9.]
 [19. 16.]
 [19. 23.]
 [19. 28.]
 [19. 31.]
 [ 1. 11.]
 [ 0. 16.]
 [ 1. 21.]
 [ 3.  6.]
 [ 1. 10.]
 [ 1. 16.]
 [ 1. 22.]
 [ 3. 26.]
 [ 7.  3.]
 [ 6.  4.]
 [ 5.  7.]
 [ 4. 11.]
 [ 4. 16.]
 [ 4. 21.]
 [ 4. 25.]
 [ 5. 28.]
 [ 6. 29.]
 [11.  1.]
 [11. 30.]
 [15.  0.]
 [15. 31.]
 [15.  0.]
 [15. 31.]
 [20.  0.]
 [19. 31.]
 [24.  3.]
 [24.  3.]
 [24.  6.]
 [24. 11.]
 [24. 16.]
 [24. 22.]
 [24. 26.]
 [24. 28.]
 [24. 29.]
 [27.  6.]
 [28.  9.]
 [28. 16.]
 [28. 22.]
 [27. 26.]
 [30. 11.]
 [30. 16.]
 [30. 21.]
 [31. 16.]]


In [7]:
# Distinguish between groups of subjects depending on the duration of T0

path = "../../Dataset"                                  # Read specific subject dataset

subj_a = []             # T0 duration = 4.1s   # List 
subj_b = []             # T0 duration = 4.2s   # List

for subject_id in range(1,110):
    
    if (subject_id == 60 or subject_id == 29 or subject_id == 30 or subject_id == 34 or subject_id == 37 or subject_id == 43 or subject_id == 64 or subject_id == 72 or subject_id == 73 or subject_id == 74 or subject_id == 76 or subject_id == 50 or subject_id == 88 or subject_id == 89 or subject_id == 92 or subject_id == 100 or subject_id == 104):   # Exclude the unwanted subjects and the testing subject (50) and the validation subject (60)
        continue
        
    file_name = os.path.join(path, './S{:003d}/S{:003d}R14.edf'.format(subject_id,subject_id))             # Read specific round of the subject
    round_data = mne.io.read_raw_edf(file_name)
    round_data.load_data()
    round_data.rename_channels(lambda s: s.strip("."))
    array_rnd_data = round_data.get_data()

    events = mne.read_annotations(file_name)         # Read annotations
    duration = getattr(events,'duration')
    
    if (duration[0] == 4.1 or duration[0] == 4.3):
        subj_a.append(subject_id)
    elif (duration[0] == 4.2):
        subj_b.append(subject_id)
    

subj_a = np.array(subj_a)               # Convert to array
subj_b = np.array(subj_b)               # Convert to array

print(subj_a)
print(subj_b)
print(len(subj_a)+len(subj_b))

[  2   4   5   6   8   9  10  11  12  13  14  15  16  17  18  19  20  23
  24  25  26  27  28  31  33  36  38  39  40  41  42  44  45  47  48  49
  51  52  53  54  55  56  58  59  62  63  67  68  69  70  75  77  78  80
  81  82  84  85  87  90  91  93  94  97  98  99 102 105 108 109]
[  1   3   7  21  22  32  35  46  57  61  65  66  71  79  83  86  95  96
 101 103 106 107]
92


In [8]:
# Subjects with bizarre total duration
subj_c = []
subj_d = []

subj_c = [29, 30]                            # Duration = 2:04 min (620 samples FFT)
subj_d = [34, 37, 64, 72, 73, 74, 76]             # Duration = 2:03 min (615 samples FFT)

print(subj_c)
print(subj_d)
print(len(subj_c)+len(subj_d))
print(len(subj_c)+len(subj_d)+len(subj_a)+len(subj_b))

[29, 30]
[34, 37, 64, 72, 73, 74, 76]
9
101


In [9]:
# Make an array for windows to select for each task

windows_a = np.zeros((30,10)) # For subjects of category a

windows_a = [[0,2,4,6,8,10,12,14,16,18],
     [21,23,25,27,29,31,33,35,37,39],
     [41,43,45,47,49,51,53,55,57,59],
     [62,64,66,68,70,72,74,76,78,80],
     [82,84,86,88,90,92,94,96,98,100],
     [103,105,107,109,111,113,115,117,119,121],
     [123,125,127,129,131,133,135,137,139,141],
     [144,146,148,150,152,154,156,158,160,162],
     [164,166,168,170,172,174,176,178,180,182],
     [185,187,189,191,193,195,197,199,201,203],
     [205,207,209,211,213,215,217,219,221,223],
     [226,228,230,232,234,236,238,240,242,244],
     [246,248,250,252,254,256,258,260,262,264],
     [267,269,271,273,275,277,279,281,283,285],
     [287,289,291,293,295,297,299,301,303,305],
     [308,310,312,314,316,318,320,322,324,326],
     [328,330,332,334,336,338,340,342,344,346],
     [349,351,353,355,357,359,361,363,365,367],
     [369,371,373,375,377,379,381,383,385,387],
     [390,392,394,396,398,400,402,404,406,408],
     [410,412,414,416,418,420,422,424,426,428],
     [431,433,435,437,439,441,443,445,447,449],
     [451,453,455,457,459,461,463,465,467,469],
     [472,474,476,478,480,482,484,486,488,490],
     [492,494,496,498,500,502,504,506,508,510],
     [513,515,517,519,521,523,525,527,529,531],
     [533,535,537,539,541,543,545,547,549,551],
     [554,556,558,560,562,564,566,568,570,572],
     [574,576,578,580,582,584,586,588,590,592],
     [595,597,599,601,603,605,607,609,611,613]]

windows_b = np.zeros((30,10)) # For subjects of category b

windows_b = [[0,2,4,6,8,10,12,14,16,18],
     [21,23,25,27,29,31,33,35,37,39],
     [41,43,45,47,49,51,53,55,57,60],
     [62,64,66,68,70,72,74,76,78,81],
     [83,85,87,89,91,93,95,97,99,102],
     [104,106,108,110,112,114,116,118,120,122],
     [125,127,129,131,133,135,137,139,141,143],
     [145,147,149,151,153,155,157,159,161,164],
     [166,168,170,172,174,176,178,180,182,185],
     [187,189,191,193,195,197,199,201,203,205],
     [208,210,212,214,216,218,220,222,224,226],
     [229,231,233,235,237,239,241,243,245,247],
     [249,251,253,255,257,259,261,263,265,268],
     [270,272,274,276,278,280,282,284,286,288],
     [290,292,294,296,298,300,302,304,306,309],
     [312,314,316,318,320,322,324,326,328,330],
     [332,334,336,338,340,342,344,346,348,351],
     [353,355,357,359,361,363,365,367,369,371],
     [374,376,378,380,382,384,386,388,390,392],
     [395,397,399,401,403,405,407,409,411,413],
     [415,417,419,421,423,425,427,429,431,434],
     [436,438,440,442,444,446,448,450,452,454],
     [456,458,460,462,464,466,468,470,472,475],
     [477,479,481,483,485,487,489,491,493,496],
     [498,500,502,504,506,508,510,512,514,517],
     [519,521,523,525,527,529,531,533,535,537],
     [539,541,543,545,547,549,551,553,555,558],
     [560,562,564,566,568,570,572,574,576,579],
     [581,583,585,587,589,591,593,595,597,600],
     [602,604,606,608,610,612,614,616,618,620]]

windows_c = np.zeros((30,10)) # For subjects of category c

windows_c = [[0,2,4,6,8,10,12,14,16,18],
     [21,23,25,27,29,31,33,35,37,39],
     [41,43,45,47,49,51,53,55,57,60],
     [62,64,66,68,70,72,74,76,78,81],
     [83,85,87,89,91,93,95,97,99,102],
     [104,106,108,110,112,114,116,118,120,122],
     [125,127,129,131,133,135,137,139,141,143],
     [145,147,149,151,153,155,157,159,161,164],
     [166,168,170,172,174,176,178,180,182,185],
     [187,189,191,193,195,197,199,201,203,205],
     [208,210,212,214,216,218,220,222,224,226],
     [229,231,233,235,237,239,241,243,245,247],
     [249,251,253,255,257,259,261,263,265,268],
     [270,272,274,276,278,280,282,284,286,288],
     [290,292,294,296,298,300,302,304,306,309],
     [312,314,316,318,320,322,324,326,328,330],
     [332,334,336,338,340,342,344,346,348,351],
     [353,355,357,359,361,363,365,367,369,371],
     [374,376,378,380,382,384,386,388,390,392],
     [395,397,399,401,403,405,407,409,411,413],
     [415,417,419,421,423,425,427,429,431,434],
     [436,438,440,442,444,446,448,450,452,454],
     [456,458,460,462,464,466,468,470,472,475],
     [477,479,481,483,485,487,489,491,493,496],
     [498,500,502,504,506,508,510,512,514,517],
     [519,521,523,525,527,529,531,533,535,537],
     [539,541,543,545,547,549,551,553,555,558],
     [560,562,564,566,568,570,572,574,576,579],
     [581,583,585,587,589,591,593,595,597,600],
     [602,604,606,608,610,612,614,616,618,619]]

windows_d = np.zeros((30,10)) # For subjects of category d

windows_d = [[0,2,4,6,8,10,12,14,16,18],
     [21,23,25,27,29,31,33,35,37,39],
     [41,43,45,47,49,51,53,55,57,60],
     [62,64,66,68,70,72,74,76,78,81],
     [83,85,87,89,91,93,95,97,99,102],
     [104,106,108,110,112,114,116,118,120,122],
     [125,127,129,131,133,135,137,139,141,143],
     [145,147,149,151,153,155,157,159,161,164],
     [166,168,170,172,174,176,178,180,182,185],
     [187,189,191,193,195,197,199,201,203,205],
     [208,210,212,214,216,218,220,222,224,226],
     [229,231,233,235,237,239,241,243,245,247],
     [249,251,253,255,257,259,261,263,265,268],
     [270,272,274,276,278,280,282,284,286,288],
     [290,292,294,296,298,300,302,304,306,309],
     [312,314,316,318,320,322,324,326,328,330],
     [332,334,336,338,340,342,344,346,348,351],
     [353,355,357,359,361,363,365,367,369,371],
     [374,376,378,380,382,384,386,388,390,392],
     [395,397,399,401,403,405,407,409,411,413],
     [415,417,419,421,423,425,427,429,431,434],
     [436,438,440,442,444,446,448,450,452,454],
     [456,458,460,462,464,466,468,470,472,475],
     [477,479,481,483,485,487,489,491,493,496],
     [498,500,502,504,506,508,510,512,514,517],
     [519,521,523,525,527,529,531,533,535,537],
     [539,541,543,545,547,549,551,553,555,558],
     [560,562,564,566,568,570,572,574,576,579],
     [581,582,583,585,586,587,589,591,593,594],
     [595,597,599,601,603,605,607,609,611,614]]

In [ ]:
# Loop through all the files, pre-process data, extract images, extract labels and save them in ndarrays

path = "../../Dataset"                                  # Read specific subject dataset

# For subjects of group a (T0 duration = 4.1 s)
for subject_id in subj_a:
        
    for round_id in range(4,15,2):
        
        file_name = os.path.join(path, './S{:003d}/S{:003d}R{:02d}.edf'.format(subject_id,subject_id,round_id))             # Read specific round of the subject
        print('./S{:003d}/S{:003d}R{:02d}.edf'.format(subject_id,subject_id,round_id))
        round_data = mne.io.read_raw_edf(file_name)
        round_data.load_data()
        round_data.rename_channels(lambda s: s.strip("."))
        array_rnd_data = round_data.get_data()
        
        # Extract labels for specific round
        # Extract events from annotations and use dictionary to specify required label - IT DEPENDS ON THE ROUND!
        # rest: 0 , left fist: 1, right fist: 2, both fists: 3, both feet: 4
        if (round_id == 4 or round_id == 8 or round_id == 12):
            dictionary = {'T0':0, 'T1':1, 'T2':2}
        
        elif (round_id == 6 or round_id == 10 or round_id == 14):
            dictionary = {'T0':0, 'T1':3, 'T2':4}
        
        ev_fr_ann, event_id = mne.events_from_annotations(round_data, event_id=dictionary)
        
        for i in range(0,30):
            round_labels[(10*i):(10*i)+10] = ev_fr_ann[i][2]   # 10 times the label because of 10 windows when extracting images of 0.4s
            # Save labels in the file to be used as training and testing labels
            with h5py.File('./../Y_training_lcomp.h5','a') as hf:
                hf["train_labels"][label_index:(label_index+10)] = round_labels[(10*i):(10*i)+10]
                label_index = label_index + 10

                    
        # TO DO: Maybe add band-pass filtering here for Delta, Mu and Beta ranges
        
        fft = mne.time_frequency.stft(array_rnd_data,64)
        
        for task in range(0,30):
            
            index = 0
            pixels = np.zeros((10,64,3))    # pixels(windows, electrodes, (R,G,B)) and each (R,G,B) corresponds to one electrode
            max_value = np.zeros(3)
            
            for window in windows_a[task]:    # Windows for each task/imagined movement
                for i in range(0,64):

                    # Extract value for Delta Band (0.5 - 4 Hz)
                    # Calculate sum of squares of absolute values
                    # all electrodes / freq_ranges 1,2,3 / window 1
                    pixels[index][i][0] = abs(fft[i][0][window])**2 + abs(fft[i][1][window])**2 + abs(fft[i][2][window])**2

                    # Extract value for Mu Band (8 - 13 Hz)
                    # Calculate sum of squares of absolute values
                    # electrode 1 / freq_ranges 4,5,6 / window 1 
                    pixels[index][i][1] = abs(fft[i][3][window])**2 + abs(fft[i][4][window])**2 + abs(fft[i][5][window])**2

                    # Extract value for Beta Band (13 - 30 Hz)
                    # Calculate sum of squares of absolute values
                    # electrode 1 / freq_ranges 7,8,9,10,11,12,13 / window 1 
                    pixels[index][i][2] = abs(fft[i][6][window])**2 + abs(fft[i][7][window])**2 + abs(fft[i][8][window])**2 + abs(fft[i][9][window])**2 + abs(fft[i][10][window])**2 + abs(fft[i][11][window])**2 + abs(fft[i][12][window])**2


                # Normalize R,G,B channels to fit the (0,1) range with the max value of each band    
                #print(np.amax(pixels))
                max_value[0] = np.amax(pixels[index,:,0]) # max value of Delta band
                max_value[1] = np.amax(pixels[index,:,1]) # max value of Mu band
                max_value[2] = np.amax(pixels[index,:,2]) # max value of Beta band

                for j in range(0,64):
                    for k in range(0,3):
                        pixels[index][j][k] = pixels[index][j][k]/max_value[k]
                        
                index = index+1
            
            
            # Assign pixel values to electrode coordinates and interpolate to create the image
            for index in range(0,10):
                interpolation[index] = griddata(electrodes,pixels[index], (image_x, image_y), method ='linear')
                
                # Save interpolated images in the file to be used as training and testing samples
                with h5py.File('./../X_training_lcomp.h5','a') as hf:
                    hf["training"][image_index] = interpolation[index]
                    image_index = image_index + 1
            
            
    traininglbs_dset.resize((1+subject_number)*1800,0)
    training_dset.resize((1+subject_number)*1800,0)
    subject_number = subject_number + 1

print("Done subjects a!")

# For subjects of group b (T0 duration = 4.2 s)
for subject_id in subj_b:
        
    for round_id in range(4,15,2):
        
        file_name = os.path.join(path, './S{:003d}/S{:003d}R{:02d}.edf'.format(subject_id,subject_id,round_id))             # Read specific round of the subject
        print('./S{:003d}/S{:003d}R{:02d}.edf'.format(subject_id,subject_id,round_id))
        round_data = mne.io.read_raw_edf(file_name)
        round_data.load_data()
        round_data.rename_channels(lambda s: s.strip("."))
        array_rnd_data = round_data.get_data()
        
        # Extract labels for specific round
        # Extract events from annotations and use dictionary to specify required label - IT DEPENDS ON THE ROUND!
        # rest: 0 , left fist: 1, right fist: 2, both fists: 3, both feet: 4
        if (round_id == 4 or round_id == 8 or round_id == 12):
            dictionary = {'T0':0, 'T1':1, 'T2':2}
        
        elif (round_id == 6 or round_id == 10 or round_id == 14):
            dictionary = {'T0':0, 'T1':3, 'T2':4}
        
        ev_fr_ann, event_id = mne.events_from_annotations(round_data, event_id=dictionary)
        
        for i in range(0,30):
            round_labels[(10*i):(10*i)+10] = ev_fr_ann[i][2]   # 10 times the label because of 10 windows when extracting images of 0.4s
            # Save labels in the file to be used as training and testing labels
            with h5py.File('./../Y_training_lcomp.h5','a') as hf:
                hf["train_labels"][label_index:(label_index+10)] = round_labels[(10*i):(10*i)+10]
                label_index = label_index + 10

        
        # TO DO: Maybe add band-pass filtering here for Delta, Mu and Beta ranges
        
        fft = mne.time_frequency.stft(array_rnd_data,64)
        
        for task in range(0,30):
            
            index = 0
            pixels = np.zeros((10,64,3))    # pixels(windows, electrodes, (R,G,B)) and each (R,G,B) corresponds to one electrode
            max_value = np.zeros(3)
            
            for window in windows_b[task]:    # Windows for each task/imagined movement
                for i in range(0,64):

                    # Extract value for Delta Band (0.5 - 4 Hz)
                    # Calculate sum of squares of absolute values
                    # all electrodes / freq_ranges 1,2,3 / window 1
                    pixels[index][i][0] = abs(fft[i][0][window])**2 + abs(fft[i][1][window])**2 + abs(fft[i][2][window])**2

                    # Extract value for Mu Band (8 - 13 Hz)
                    # Calculate sum of squares of absolute values
                    # electrode 1 / freq_ranges 4,5,6 / window 1 
                    pixels[index][i][1] = abs(fft[i][3][window])**2 + abs(fft[i][4][window])**2 + abs(fft[i][5][window])**2

                    # Extract value for Beta Band (13 - 30 Hz)
                    # Calculate sum of squares of absolute values
                    # electrode 1 / freq_ranges 7,8,9,10,11,12,13 / window 1 
                    pixels[index][i][2] = abs(fft[i][6][window])**2 + abs(fft[i][7][window])**2 + abs(fft[i][8][window])**2 + abs(fft[i][9][window])**2 + abs(fft[i][10][window])**2 + abs(fft[i][11][window])**2 + abs(fft[i][12][window])**2


                # Normalize R,G,B channels to fit the (0,1) range with the max value of each band    
                #print(np.amax(pixels))
                max_value[0] = np.amax(pixels[index,:,0]) # max value of Delta band
                max_value[1] = np.amax(pixels[index,:,1]) # max value of Mu band
                max_value[2] = np.amax(pixels[index,:,2]) # max value of Beta band

                for j in range(0,64):
                    for k in range(0,3):
                        pixels[index][j][k] = pixels[index][j][k]/max_value[k]
                        
                index = index+1
                
            # Assign pixel values to electrode coordinates and interpolate to create the image
            for index in range(0,10):
                interpolation[index] = griddata(electrodes,pixels[index], (image_x, image_y), method ='linear')
                
                # Save interpolated images in the file to be used as training and testing samples
                with h5py.File('./../X_training_lcomp.h5','a') as hf:
                    hf["training"][image_index] = interpolation[index]
                    image_index = image_index + 1
                    
    traininglbs_dset.resize((1+subject_number)*1800,0)
    training_dset.resize((1+subject_number)*1800,0)
    subject_number = subject_number + 1
                    
print("Done subjects b!")


# For subjects of group c (Total duration = 2:04 min)
for subject_id in subj_c:
        
    for round_id in range(4,15,2):
        
        file_name = os.path.join(path, './S{:003d}/S{:003d}R{:02d}.edf'.format(subject_id,subject_id,round_id))             # Read specific round of the subject
        print('./S{:003d}/S{:003d}R{:02d}.edf'.format(subject_id,subject_id,round_id))
        round_data = mne.io.read_raw_edf(file_name)
        round_data.load_data()
        round_data.rename_channels(lambda s: s.strip("."))
        array_rnd_data = round_data.get_data()
        
        # Extract labels for specific round
        # Extract events from annotations and use dictionary to specify required label - IT DEPENDS ON THE ROUND!
        # rest: 0 , left fist: 1, right fist: 2, both fists: 3, both feet: 4
        if (round_id == 4 or round_id == 8 or round_id == 12):
            dictionary = {'T0':0, 'T1':1, 'T2':2}
        
        elif (round_id == 6 or round_id == 10 or round_id == 14):
            dictionary = {'T0':0, 'T1':3, 'T2':4}
        
        ev_fr_ann, event_id = mne.events_from_annotations(round_data, event_id=dictionary)
        
        for i in range(0,30):
            round_labels[(10*i):(10*i)+10] = ev_fr_ann[i][2]   # 10 times the label because of 10 windows when extracting images of 0.4s
            # Save labels in the file to be used as training and testing labels
            with h5py.File('./../Y_training_lcomp.h5','a') as hf:
                hf["train_labels"][label_index:(label_index+10)] = round_labels[(10*i):(10*i)+10]
                label_index = label_index + 10

        
        # TO DO: Maybe add band-pass filtering here for Delta, Mu and Beta ranges
        
        fft = mne.time_frequency.stft(array_rnd_data,64)
        
        for task in range(0,30):
            
            index = 0
            pixels = np.zeros((10,64,3))    # pixels(windows, electrodes, (R,G,B)) and each (R,G,B) corresponds to one electrode
            max_value = np.zeros(3)
            
            for window in windows_c[task]:    # Windows for each task/imagined movement
                for i in range(0,64):

                    # Extract value for Delta Band (0.5 - 4 Hz)
                    # Calculate sum of squares of absolute values
                    # all electrodes / freq_ranges 1,2,3 / window 1
                    pixels[index][i][0] = abs(fft[i][0][window])**2 + abs(fft[i][1][window])**2 + abs(fft[i][2][window])**2

                    # Extract value for Mu Band (8 - 13 Hz)
                    # Calculate sum of squares of absolute values
                    # electrode 1 / freq_ranges 4,5,6 / window 1 
                    pixels[index][i][1] = abs(fft[i][3][window])**2 + abs(fft[i][4][window])**2 + abs(fft[i][5][window])**2

                    # Extract value for Beta Band (13 - 30 Hz)
                    # Calculate sum of squares of absolute values
                    # electrode 1 / freq_ranges 7,8,9,10,11,12,13 / window 1 
                    pixels[index][i][2] = abs(fft[i][6][window])**2 + abs(fft[i][7][window])**2 + abs(fft[i][8][window])**2 + abs(fft[i][9][window])**2 + abs(fft[i][10][window])**2 + abs(fft[i][11][window])**2 + abs(fft[i][12][window])**2


                # Normalize R,G,B channels to fit the (0,1) range with the max value of each band    
                #print(np.amax(pixels))
                max_value[0] = np.amax(pixels[index,:,0]) # max value of Delta band
                max_value[1] = np.amax(pixels[index,:,1]) # max value of Mu band
                max_value[2] = np.amax(pixels[index,:,2]) # max value of Beta band

                for j in range(0,64):
                    for k in range(0,3):
                        pixels[index][j][k] = pixels[index][j][k]/max_value[k]
                        
                index = index+1
                
            # Assign pixel values to electrode coordinates and interpolate to create the image
            for index in range(0,10):
                interpolation[index] = griddata(electrodes,pixels[index], (image_x, image_y), method ='linear')
                
                # Save interpolated images in the file to be used as training and testing samples
                with h5py.File('./../X_training_lcomp.h5','a') as hf:
                    hf["training"][image_index] = interpolation[index]
                    image_index = image_index + 1
                    
    traininglbs_dset.resize((1+subject_number)*1800,0)
    training_dset.resize((1+subject_number)*1800,0)
    subject_number = subject_number + 1
                    
print("Done subjects c!")

# For subjects of group d (Total duration = 2:03 min)
for subject_id in subj_d:
        
    for round_id in range(4,15,2):
        
        file_name = os.path.join(path, './S{:003d}/S{:003d}R{:02d}.edf'.format(subject_id,subject_id,round_id))             # Read specific round of the subject
        print('./S{:003d}/S{:003d}R{:02d}.edf'.format(subject_id,subject_id,round_id))
        round_data = mne.io.read_raw_edf(file_name)
        round_data.load_data()
        round_data.rename_channels(lambda s: s.strip("."))
        array_rnd_data = round_data.get_data()
        
        # Extract labels for specific round
        # Extract events from annotations and use dictionary to specify required label - IT DEPENDS ON THE ROUND!
        # rest: 0 , left fist: 1, right fist: 2, both fists: 3, both feet: 4
        if (round_id == 4 or round_id == 8 or round_id == 12):
            dictionary = {'T0':0, 'T1':1, 'T2':2}
        
        elif (round_id == 6 or round_id == 10 or round_id == 14):
            dictionary = {'T0':0, 'T1':3, 'T2':4}
        
        ev_fr_ann, event_id = mne.events_from_annotations(round_data, event_id=dictionary)
        
        for i in range(0,30):
            round_labels[(10*i):(10*i)+10] = ev_fr_ann[i][2]   # 10 times the label because of 10 windows when extracting images of 0.4s
            # Save labels in the file to be used as training and testing labels
            with h5py.File('./../Y_training_lcomp.h5','a') as hf:
                hf["train_labels"][label_index:(label_index+10)] = round_labels[(10*i):(10*i)+10]
                label_index = label_index + 10

        
        # TO DO: Maybe add band-pass filtering here for Delta, Mu and Beta ranges
        
        fft = mne.time_frequency.stft(array_rnd_data,64)
        
        for task in range(0,30):
            
            index = 0
            pixels = np.zeros((10,64,3))    # pixels(windows, electrodes, (R,G,B)) and each (R,G,B) corresponds to one electrode
            max_value = np.zeros(3)
            
            for window in windows_d[task]:    # Windows for each task/imagined movement
                for i in range(0,64):

                    # Extract value for Delta Band (0.5 - 4 Hz)
                    # Calculate sum of squares of absolute values
                    # all electrodes / freq_ranges 1,2,3 / window 1
                    pixels[index][i][0] = abs(fft[i][0][window])**2 + abs(fft[i][1][window])**2 + abs(fft[i][2][window])**2

                    # Extract value for Mu Band (8 - 13 Hz)
                    # Calculate sum of squares of absolute values
                    # electrode 1 / freq_ranges 4,5,6 / window 1 
                    pixels[index][i][1] = abs(fft[i][3][window])**2 + abs(fft[i][4][window])**2 + abs(fft[i][5][window])**2

                    # Extract value for Beta Band (13 - 30 Hz)
                    # Calculate sum of squares of absolute values
                    # electrode 1 / freq_ranges 7,8,9,10,11,12,13 / window 1 
                    pixels[index][i][2] = abs(fft[i][6][window])**2 + abs(fft[i][7][window])**2 + abs(fft[i][8][window])**2 + abs(fft[i][9][window])**2 + abs(fft[i][10][window])**2 + abs(fft[i][11][window])**2 + abs(fft[i][12][window])**2


                # Normalize R,G,B channels to fit the (0,1) range with the max value of each band    
                #print(np.amax(pixels))
                max_value[0] = np.amax(pixels[index,:,0]) # max value of Delta band
                max_value[1] = np.amax(pixels[index,:,1]) # max value of Mu band
                max_value[2] = np.amax(pixels[index,:,2]) # max value of Beta band

                for j in range(0,64):
                    for k in range(0,3):
                        pixels[index][j][k] = pixels[index][j][k]/max_value[k]
                        
                index = index+1
                
            # Assign pixel values to electrode coordinates and interpolate to create the image
            for index in range(0,10):
                interpolation[index] = griddata(electrodes,pixels[index], (image_x, image_y), method ='linear')
                
                # Save interpolated images in the file to be used as training and testing samples
                with h5py.File('./../X_training_lcomp.h5','a') as hf:
                    hf["training"][image_index] = interpolation[index]
                    image_index = image_index + 1
                    
    traininglbs_dset.resize((1+subject_number)*1800,0)
    training_dset.resize((1+subject_number)*1800,0)
    subject_number = subject_number + 1
    
f.close()   

print("Done subjects d!")
print("DONE.")


./S002/S002R04.edf
./S002/S002R06.edf
./S002/S002R08.edf
./S002/S002R10.edf
./S002/S002R12.edf
./S002/S002R14.edf
./S004/S004R04.edf
./S004/S004R06.edf
./S004/S004R08.edf
./S004/S004R10.edf
./S004/S004R12.edf
./S004/S004R14.edf
./S005/S005R04.edf
./S005/S005R06.edf
./S005/S005R08.edf
./S005/S005R10.edf
./S005/S005R12.edf
./S005/S005R14.edf
./S006/S006R04.edf
./S006/S006R06.edf
./S006/S006R08.edf
./S006/S006R10.edf
./S006/S006R12.edf
./S006/S006R14.edf
./S008/S008R04.edf
./S008/S008R06.edf
./S008/S008R08.edf
./S008/S008R10.edf
./S008/S008R12.edf
./S008/S008R14.edf
./S009/S009R04.edf
./S009/S009R06.edf
./S009/S009R08.edf
./S009/S009R10.edf
./S009/S009R12.edf
./S009/S009R14.edf
./S010/S010R04.edf
./S010/S010R06.edf
./S010/S010R08.edf
./S010/S010R10.edf
./S010/S010R12.edf
./S010/S010R14.edf
./S011/S011R04.edf
./S011/S011R06.edf
./S011/S011R08.edf
./S011/S011R10.edf
./S011/S011R12.edf
./S011/S011R14.edf
./S012/S012R04.edf
./S012/S012R06.edf
./S012/S012R08.edf
./S012/S012R10.edf
./S012/S012R

./S003/S003R14.edf
./S007/S007R04.edf
./S007/S007R06.edf
./S007/S007R08.edf
./S007/S007R10.edf
./S007/S007R12.edf
./S007/S007R14.edf
./S021/S021R04.edf
./S021/S021R06.edf
./S021/S021R08.edf
./S021/S021R10.edf
./S021/S021R12.edf
./S021/S021R14.edf
./S022/S022R04.edf
./S022/S022R06.edf
./S022/S022R08.edf
./S022/S022R10.edf
./S022/S022R12.edf
./S022/S022R14.edf
./S032/S032R04.edf
./S032/S032R06.edf
./S032/S032R08.edf
